## Q1. Running Elastic

In [1]:
from elasticsearch import Elasticsearch

In [2]:
es_client = Elasticsearch('http://localhost:9200')

In [3]:
es_client.info()

ObjectApiResponse({'name': 'bad94fae3936', 'cluster_name': 'docker-cluster', 'cluster_uuid': '1Qt29qX6TGq6HVF6t9qqww', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [7]:
es_client.info()['version']['build_hash']

'42f05b9372a9a4a470db3b52817899b99a76ee73'

What's the `version.build_hash` value?

`42f05b9372a9a4a470db3b52817899b99a76ee73`




In [8]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2. Indexing the data

In [9]:
index_setting = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course_questions"

es_client.indices.create(index=index_name, body=index_setting)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course_questions/MNUDVbdEQCa9ElQb8k8Bww] already exists')

In [10]:
from tqdm.auto import tqdm

In [11]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document = doc)

  0%|          | 0/948 [00:00<?, ?it/s]

Which function do you use for adding your data to elastic?
 
* `index`

## Q3. Searching

In [20]:
def elastic_search(query):

    search_query ={
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    } 
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    #result_docs = []
    
    #for hit in response['hits']['hits']:
    #    result_docs.append(hit['_source'])
    
    return response

In [26]:
query  = "How do I execute a command in a running docker container?"
search_result = elastic_search(query) 

In [34]:
search_result['hits']['hits'][0]['_score']

84.17781

What's the score for the top ranking result?

* 84.05

## Q4. Filtering

In [44]:
def elastic_search(query):

    search_query ={
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    } 
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    #result_docs = []
    
    #for hit in response['hits']['hits']:
    #    result_docs.append(hit['_source'])
    
    return response

In [45]:
query = "How do I debug a docker container?"
search_result = elastic_search(query) 

In [60]:
search_result['hits']['hits'][2] 

{'_index': 'course_questions',
 '_id': 'VuNMsZEBzIAJugZmquu4',
 '_score': 51.134113,
 '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
  'course': 'machine-learning-zoomcamp'}}

Return 3 results. What's the 3rd question returned by the search engine?

* How do I copy files from a different folder into docker container’s working directory?


In [68]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [69]:
context_pieces = []

for hit in search_result['hits']['hits']:
    doc = hit['_source']
    context_piece = context_template.format(**doc)
    context_pieces.append(context_piece)

context = '\n\n'.join(context_pieces)

In [70]:
prompt = prompt_template.format(question=query, context=context)

In [71]:
len(prompt)

1376

What's the length of the resulting prompt? (use the len function)

* 1462

## Q6. Tokens

In [74]:
import tiktoken

In [76]:
print(prompt[:100])

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.



In [79]:
encoding_name = "cl100k_base"  # Typically used for models like GPT-4
encoding = tiktoken.get_encoding(encoding_name)
 

In [80]:
len(encoding.encode(prompt))

306

Use the encode function. How many tokens does our prompt have?

* 322

In [81]:
tokens = encoding.encode(prompt)[:10]
tokens

[2675, 2351, 264, 3388, 12917, 18328, 13, 22559, 279, 91022]

In [83]:
encoding.decode_single_token_bytes(tokens[5]) 

b' assistant'